In [7]:
import pandas as pd

In [8]:
from scapy.all import *

Wczytanie pliku pcap
Przechodzimy po wszystkich pakietach

In [ ]:
from scapy.layers.inet import IP, TCP

scapy_cap = rdpcap('test_dir/trick_bot.pcap')
packet_tab = []
event_449 = 0
event_447 = 0
for packet in scapy_cap:
    if TCP in packet:
        ip_src = ""
        ip_dst = ""
        if IP in packet:
            ip_src=packet[IP].src
            ip_dst=packet[IP].dst

        tcp_time=packet[TCP].time
        tcp_sport=packet[TCP].sport
        tcp_dport=packet[TCP].dport
        packet_tab.append([str(tcp_time),tcp_sport,tcp_dport, ip_src, ip_dst])
        if tcp_sport == 449 or tcp_dport ==449:
            event_449 += 1
        if tcp_sport == 447 or tcp_dport ==447:
            event_447 += 1

if event_449 > 100 and event_447 > 50:
    print("alert")

df = pd.DataFrame(packet_tab, columns=['time','src_port', 'dst_port', 'ip_src', 'ip_dst'])

In [44]:
df.groupby(["dst_port"]).size().rename("amount").reset_index()

,dst_port,amount
0,80,484
1,443,189
2,445,2
3,447,3475
4,448,18
...,...,...
149,49568,10
150,49569,12
151,49570,10
152,49571,10


In [51]:
import matplotlib.pyplot as plt
connections = df.groupby(["dst_port"]).size().rename("amount").reset_index()

In [56]:
connections = connections[connections.amount>100]

Porty z największą liczbą połączeń:

In [60]:
connections.sort_values('amount', ascending=False)

,dst_port,amount
119,49528,6557
3,447,3475
37,49187,1330
5,449,743
45,49195,650
15,49165,500
0,80,484
47,49197,371
1,443,189
109,49518,187


In [78]:
connections = connections[connections.dst_port<1024]
connections

,dst_port,amount
0,80,484
1,443,189
3,447,3475
5,449,743


Połączenia na portach 80 i 443 są traktowane jako bezpieczne. Nastomiast obawę może wzbudzć ilośc połączeń na portach 447, 449.

In [68]:
trusted_ports = [80, 443]

In [79]:
connections[~connections.dst_port.isin(trusted_ports)]

,dst_port,amount
3,447,3475
5,449,743
